In [ ]:
#KERAS introduction & implementation
"""
Keras'ta bir sinir ağı oluşturmanın çeşitli adımları vardır. Burada sırasıyla bu adımlardan ve adımlarla ilgili bazı olgulardan 
bahsedeceğiz.

1) Öncelikle bir model nesnesi oluşturulmalıdır. tensorflow.keras modülü içerisinde çeşitli model sınıfları bulunmaktadır. 
En çok kullanılan model sınıfı Sequential isimli sınıftır. Tüm model sınıfları Model isimli sınıftan türetilmiştir. Sequential 
modelde ağa her eklenen katman sona eklenir. Böylece ağ katmanların sırasıyla eklenmesiyle oluşturulur. Sequential nesnesi 
yaratılırken name parametresiyle modele bir isim de verilebilir. Örneğin:

from tensorflow.keras import Sequential

model = Sequential(name='Sample')

Aslında Sequential nesnesi yaratılırken katmanlar belirlendiyse layers parametresiyle bu katmanlar da verilebilmektedir. Ancak
sınıfın tipik kullanımında katmanlar daha sonra izleyen maddelerde ele alınacağı gibi sırasıyla eklenmektedir.

2) Model nesnesinin yaratılmasından sonra katman nesnelerinin oluşturulup model nesnesine eklenmesi gerekir. Keras'ta farklı 
gereksinimler için farklı katman sınıfları bulundurulmuştur. En çok kullanılan katman sınıfı tensorflow.keras.layers modülündeki 
Dense sınıfıdır. Dense bir katman modele eklendiğinde önceki katmandaki tüm nöronların çıktıları eklenen katmandaki nöronların 
hepsine girdi yapılmaktadır. Bu durumda örneğin önceki katmanda k tane nöron varsa biz de modele n tane nörondan oluşan bir Dense 
katman ekliyorsak bu durumda modele (k * n + n) tane yeni parametre (yani tahmin edilmesi gereken parametre) eklemiş oluruz. Burada 
k * n tane ayarlanması gereken w (ağırlık) değerleri ve n tane de ayarlanması gereken bias değerleri söz konusudur. Bir nörondaki 
w (ağırlık) değerlerinin o nörona giren nöron sayısı kadar olduğuna ve bias değerlerinin her nöron için bir tane olduğuna dikkat
ediniz.

Dense sınıfının __init__ metodunun ilk parametresi eklenecek katmandaki nöron sayısını belirtir. İkinci parametre olan activation 
parametresi o katmandaki tüm nöronların aktivasyon fonksiyonlarının ne olacağını belirtmektedir. Bu parametreye aktivasyon 
fonksiyonları birer yazı biçiminde isimsel olarak girilebilir. Ya da tensorflow.keras.activations modülündeki fonksiyonlar 
olarak girilebilir. Örneğin:

from tensorflow.keras.layers import Dense

layer = Dense(100, activation='relu')

ya da örneğin:

from tensorflow.keras.activations import relu

layer = Dense(100, activation=relu)

Dense fonksiyonun use_bias parametresi default durumda True biçimdedir. Bu parametre katmandaki nöronlarda "bias" değerinin 
kullanılıp kullanılmayacağını belirtmektedir. Metodun kernel_initializer parametresi katmandaki nöronlarda kullanılan w 
parametrelerinin ilkdeğerlerinin rastgele biçimde hangi algoritmayla oluşturulacağını belirtmektedir. Bu parametrenin default
değeri "glorot_unfiorm" biçimindedir. Metodun bias_initializer parametresi ise katmandaki nöronların "bias" değerlerinin başlangıçta 
nasıl alınacağını belirtmektedir. Bu parametrenin default değeri de "zero" biçimdedir. Yani bias değerleri başlangıçta 0 
durumundadır. Dense sınıfının __init__ metodunun diğer parametreleri hakkında bu aşamada bilgi vermeyeceğiz. 

Keras'ta Sequential modelde girdi katmanı programcı tarafından yaratılmaz. İlk saklı katman yaratılırken girdi katmanındaki 
nöron sayısı input_dim parametresiyle ya da input_shape parametresiyle belirtilmektedir. input_dim tek boyutlu girdiler için 
input_shape ise çok boyutlu girdiler için kullanılmaktadır. Örneğin:

layer = Dense(100, activation='relu', input_dim=8) #8 sutun sayisina denk geliyor demek

Tabii input_dim ya da input_shape parametrelerini yalnızca ilk saklı katmanda kullanabiliriz. Genel olarak ağın girdi katmanında
dataset_x'teki sütun sayısı kadar nöron olacağına göre ilk katmandaki input_dim parametresini aşağıdaki gibi de girebiliriz:

layer = Dense(100, activation='relu', input_dim=training_dataset_x.shape[1])
    
Aslında Keras'ta girdi katmanı için tensorflow.keras.layers modülünde Input isminde bir karman da kullanılmaktadır. Tenseoflow'un
yeni versiyonlarında girdi katmanının Input katmanı ile oluşturulması istenmektedir. Aksi takdirde bu yeni versiyonlar uyarı 
vermektedir. Girdi katmanını Input isimli katman sınıfıyla oluştururken bu Input sınıfının __init__ metodunun birinci parametresi
bir demet biçiminde (yani sahpe olarak) girilmelidir. Örneğin:

input = Input((8, ))

Burada 8 nöronşuk bir girdi katmanı oluşturulmuştur. Yukarıda da belirttiğimiz gibi eskiden ilk saklı katmanda girdi katmanı 
belirtiliyordu. Ancak Tensorflow kütüphanesinin yeni verisyonlarında ilk saklı katmanda girdi katmanının belirtilmesi artık 
uyarıya (warning) yol açmaktadır. Önceki kurslarda biz girdi katmanını ilk saklı katmanda belirtiyorduk. Ancak artık bu kursumuzda
girdi katmanını ayrı bir Input nesnesi ile oluşturacağız. 

Her katmana istersek name parametresi ile bir isim de verebiliriz. Bu isimler model özeti alınırken ya da katmanlara erişilirken 
kullanılabilmektedir. Örneğin:

layer = Dense(100, activation='relu',  name='Hidden-1')

Oluşturulan katman nesnesinin model nesnesine eklenmesi için Sequential sınıfının add metodu kullanılmaktadır. Örneğin:

input = Input((8, ))
model.add(input)
layer = Dense(100, activation='relu',  name='Hidden-1')
model.add(layer)

Programcılar genellikle katman nesnesinin yaratılması ve eklenmesini tek satırda aşağıdaki gibi yaparlar:

model.add(Dense(100, activation='relu', input_dim=9, name='Hidden-1'))

3) Modele katmanlar eklendikten sonra bir özet bilgi yazdırılabilir. Bu işlem Sequential sınıfının summary isimli metoduyla 
yapılmaktadır.

Yukarıda da belirttiğimiz gibi bir katmandaki "eğitilebilir (trainable)" parametrelerin sayısı şöyle hesaplanmaktadır: Önceki 
katmanın çıktısındaki nöron sayısı, bu katmana dense biçimde bağlandığına göre toplamda "önceki katmandaki nöron sayısı * 
bu katmandaki nöron sayısı" kadar w parametresi ayarlanacaktır. Öte yandan bu katmandaki nöronların her birinde bir "bias" 
değeri olduğuna göre bu değerler de bu sayıya toplanmalıdır. O zaman önceki katmandaki nöron sayısı k, bu katmandaki nöron 
sayısı n ise eğitilebilir parametrelerin sayısı k * n + n tane olur. Aşağıdaki modeli inceleyiniz:

model = Sequential(name='Diabetes')

model.add(Input((training_dataset_x.shape[1],)))
model.add(Dense(16, activation='relu', name='Hidden-1'))
model.add(Dense(16, activation='relu', name='Hidden-2'))
model.add(Dense(1, activation='sigmoid', name='Output'))
model.summary()

B modelde bir girdi katmanı, iki saklı katman (biz bunlara ara katman da diyeceğiz) bir de çıktı katmanı vardır. 
summary metodundan elde edilen çıktı şöyledir.

Model: "Diabetes"
┌─────────────────────────────────┬────────────────────────┬───────────────┐
│ Layer (type)                    │ Output Shape           │       Param # │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden-1 (Dense)                │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden-2 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘
Total params: 433 (1.69 KB)
Trainable params: 433 (1.69 KB)
Non-trainable params: 0 (0.00 B)
Trainable params: 433 (1.69 KB)
Non-trainable params: 0 (0.00 B)

Burada ağımızdaki girdi katmanında 8 nöron olduğuna göre ve ilk saklı katmanda da 16 nöron olduğuna göre ilk saklı katmana
8 * 16 nöron girmektedir. Öte yandan her nöronun bir tane bias değeri de olduğuna göre ilk katmandaki tahmin ayarlanması 
gereken parametrelerin (trainable parameters) sayısı 8 * 16 + 16 = 144 tanedir. İkinci saklı katmana 16 nöron dense biçimde
bağlanmıştır. O halde ikinci saklı katmandaki ayarlanması gereken parametreler toplamda 16 * 16 + 16 = 272 tanedir. Modelimizin
çıktı katmanında 1 nöron vardır. Önceki katmanın 16 çıkışı olduğuna göre bu çıktı katmanında 16 * 1 + 1 = 17 tane ayarlanması
gereken parametre vardır. 

Ağın saklı katmanlarında en çok kullanılan aktivasyon fonksiyonu "relu" isimli fonksiyondur. İkili sınıflandırma problemlerinde 
çıktı katmanı tek nörondan oluşur ve bu katmandaki aktivasyon fonksiyonu "sigmoid" fonksiyonu olur. Sigmoid fonksiyonu 0 ile
1 arasında bir değer vermektedir. Aktivasyon fonksiyonlarını izleyen paragraflarda ele alacağız.

Aşağıdaki örnekte "dibates" veri kümesi üzerinde ikili sınıflandırma problemi için bir sinir ağı oluşturulmuştur.
"""

In [1]:
#bu kodda 16+16 seklinde 2 hidden katmandan olusan bir neural network var.Cikistada bir noron output norondan olusan bir katman ornegi var.
import pandas as pd

df = pd.read_csv('diabetes.csv')

from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy='mean', missing_values=0)
df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = si.fit_transform(df[['Glucose', 'BloodPressure', 
        'SkinThickness', 'Insulin', 'BMI']])

#numpya donusum ve datasetlere ayirma kismi
#Bu işlemlerin amacı, genellikle makine öğrenmesi modellerinin eğitilmesi için gerekli olan bağımsız değişkenler (X) ve bağımlı değişken (y) ayrıştırmasını yapmaktır. Örneğin, bir doğrusal regresyon problemi için, dataset_x bağımsız değişkenleri (özellikler) içerirken, dataset_y tahmin edilmesi gereken bağımlı değişkeni (hedef değişkeni) içerir
dataset = df.to_numpy()
dataset_x = dataset[:, :-1]
dataset_y = dataset[:, -1]

from sklearn.model_selection import train_test_split
training_dataset_x, test_dataset_x, training_dataset_y, test_dataset_y = train_test_split(dataset_x, dataset_y, test_size=0.2)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

model = Sequential(name='Diabetes') #kerasin model nesnesi yaratilmali

#katmanlarin olusturulmasi : noron sayisini belirlerken fazla katman olursa overfitting olabilir dikkatli olmak gerekir.
model.add(Input((training_dataset_x.shape[1],)))
model.add(Dense(16, activation='relu', name='Hidden-1')) #1 hidden katman nesnesi 16 tane norondan olusuyor demek ki.(1 .katmanda [8*16(W) degeri + 16(Bayes)=144 tane parametre degeri var]
model.add(Dense(16, activation='relu', name='Hidden-2')) #(2 .hidden katmanda [16*16(W) degeri + 16(Bayes)=272 tane parametre degeri var]
model.add(Dense(1, activation='sigmoid', name='Output')) #(Output katmanda [16*1(W) degeri + 1(Bayes)=17 tane parametre degeri var genelde sigmoid fonksiyonu kullanilir]
model.summary()

Model: "Diabetes"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-1 (Dense)            (None, 16)                144       
                                                                 
 Hidden-2 (Dense)            (None, 16)                272       
                                                                 
 Output (Dense)              (None, 1)                 17        
                                                                 
Total params: 433 (1.69 KB)
Trainable params: 433 (1.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
"""
4) Model oluşturulduktan sonra modelin derlenmesi (compile edilmesi) gerekir. Buradaki "derleme" makine diline dönüştürme 
anlamında bir terim değildir. Eğitim için bazı belirlemelerin yapılması anlamına gelmektedir. Bu işlem Sequential sınıfının 
compile isimli metoduyla yapılmaktadır. Modelin compile metoduyla derlenmesi sırasında en önemli iki parametre "loss fonksiyonu" 
ve "optimizasyon algoritması"dır. Eğitim sırasında ağın ürettiği değerlerin gerçek değerlere yaklaştırılması için w ve bias 
değerlerinin nasıl güncelleneceğine ilişkin algoritmalara "optimizasyon algoritmaları" denilmektedir. Matematiksel optimizasyon 
işlemlerinde belli bir fonksiyonun minimize edilmesi istenir. İşte minimize edilecek bu fonksiyona da "loss fonksiyonu" 
denilmektedir. Başka bir deyişle optimizasyon algoritması loss fonksiyonun değerini minimize edecek biçimde işlem yapan 
algoritmadır. Yani optimizasyon algoritması loss fonksiyonunu minimize etmek için yapılan işlemleri temsil etmektedir. 

Loss fonksiyonları ağın ürettiği değerlerle gerçek değerler arasındaki farklılığı temsil eden fonksiyonlardır. Loss fonksiyonları 
genel olarak iki girdi alıp bir çıktı vermektedir. Loss fonksiyonunun girdileri gerçek değerler ile ağın ürettiği değerlerdir. 
Çıktı değeri ise aradaki farklığı belirten bir değerdir. Eğitim sırasında gitgide loss fonksiyonun değerinin düşmesini bekleriz. 
Tabii loss değerinin düşmesi aslında ağın gerçek değerlere daha yakın değerler üretmesi anlamına gelmektedir. loss fonksiyonları 
çıktının biçimine yani problemin türüne bağlı olarak seçilmektedir. Örneğin ikili sınıflandırma problemleri için "binary cross-entropy", 
çoklu sınıflandırma problemleri için "categorical cross-entropy", lojistik olmayan regresyon problemleri için "mean squared error" 
isimli loss fonksiyonları tercih edilmektedir. 

Optimizasyon algoritmaları aslında genel yapı olarak birbirlerine benzemektedir. Pek çok problemde bu algoritmaların çoğu benzer 
performans göstermektedir. En çok kullanılan optimizasyon algoritmaları "rmsprop", "adam" ve "sgd" algoritmalarıdır. Bu algoritmalar 
"gradient descent" denilen genel optimizasyon yöntemini farklı biçimlerde uygulamaktadır. Optimizasyon algoritmaları kursumuzun 
ilerleyen bölümlerinde ele alınacaktır.

compile metodunda optimizasyon algoritması bir yazı olarak ya da tensorflow.keras.optimizers modülündeki sınıflar türünden bir 
sınıf nesnesi olarak girilebilmektedir. Örneğin:

model.compile(optimizer='rmsprop', ...)

Örneğin:

from tensorflow.keras.optimizers import RMSprop

rmsprop = RMSprop()

model.compile(optimizer=rmsprop, ...)

Tabii optimizer parametresinin bir sınıf nesnesi olarak girilmesi daha detaylı belirlemelerin yapılmasına olanak sağlamaktadır.
Optimizasyon işlemlerinde bazı parametrik değerler vardır. Bunlara makine öğrenmesinde "üst düzey parametreler (hyper parameters)"
denilmektedir. İşte optimizasyon algoritması bir sınıf nesnesi biçiminde verilirse bu sınıfın __init__ metodunda biz bu üst 
düzey parametreleri istediğimiz gibi belirleyebiliriz. Eğer optimizasyon algoritması yazısal biçimde verilirse bu üst düzey 
parametreler default değerlerle kullanılmaktadır. optimzer parametresinin default değeri "rmsprop" biçimindedir. Yani biz bu 
parametre için değer girmezsek default optimazyon algoritması "rmsprop" olarak alınacaktır. 

loss fonksiyonu compile metoduna yine isimsel olarak ya da tensorflow.keras.losses modülündeki sınıflar türünden sınıf nesleri 
biçiminde ya da doğrudan fonksiyon olarak girilebilmektedir. loss fonksiyonları kısa ya da uzun isim olarak yazısal biçimde 
kullanılabilmektedir. Tipik loss fonksiyon isimleri şunlardır:

'mean_squared_error' ya da 'mse'
'mean_absolute_error' ya da 'mae'
'mean_absolute_percentage_error' ya da 'mape'
'mean_squared_logarithmic_error' ya da 'msle'
'categorical_crossentropy'
'binary_crossentropy'

tensorflow.keras.losses modülündeki fonksiyonlar da kısa ve uzun isimli olarak bulunabilmektedir:

tensorflow.keras.losses.mean_squared_error ya da tensorflow.keras.losses.mse
tensorflow.keras.losses.mean_absolute_error ya da tensorflow.keras.losses.mae
tensorflow.keras.losses.mean_absolute_percentage_error ya da tensorflow.keras.losses.mape
tensorflow.keras.losses.mean_squared_logarithmic_error ya da tensorflow.keras.losses.msle
tensorflow.keras.losses.categorical_crossentropy
tensorflow.keras.losses.binary_crossentropy

Bu durumda compile metodu örnek bir biçimde şöyle çağrılabilir:

model.compile(optimizer='rmsprop', loss='binary_crossentropy')

compile metodunun üçüncü önemli parametresi "metrics" isimli parametredir. metrics parametresi bir liste ya da demet olarak 
girilir. metrics parametresi daha sonra açıklanacak olan "sınama (validation)" işlemi için kullanılacak fonksiyonları 
belirtmektedir. Sınamada her zaman zaten bizim loss fonksiyonu olarak belirttiğimiz fonksiyon kullanılmaktadır. metrics 
parametresinde ilave fonksiyonlar da girilebilmektedir. Örneğin ikili sınıflandırma problemleri için tipik olarak "binary_accuracy"
denilen metrik fonksiyon, çoklu sınıflandırma için "categorical_accuracy" denilen metrik fonksiyon ve lojistik olmayan regresyon
problemleri için de "mean_absolute_error" isimli metrik fonksiyon sıklıkla kullanılmaktadır. Örneğin ikili sınıflandırma 
problemi için biz eğitim sırasında "loss" değerinin yanı sıra "binary_accuracy" değerini de elde etmek isteyelim. Bu durumda 
compile metodunu şöyle çağırmalıyız:

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])

5) Model derlenip çeşitli belirlemeler yapıldıktan sonra artık gerçekten eğitim aşamasına geçilir. Eğitim süreci Sequential 
sınıfının fit metoduyle yapılmaktadır. fit metodunun en önemli parametresi ilk iki parametre olan x ve y veri kümeleridir. 
Biz burada training_dataset_x ve training_dataset_y verilerini fit metodunun ilk iki paramtresine geçirmeliyiz. 

fit metodunun önemli bir parametresi batch_size isimli parametredir. Eğitim işlemi aslında satır satır değil batch batch 
yapılmaktadır. batch bir grup satıra denilmektedir. Yani ağa bir grup satır girdi olarak verilir. Ağdan bir grup çıktı elde 
edilir. Bu bir grup çıktı ile bu çıktıların gerçek değerleri loss fonksiyonuna sokulur ve optimizasyon algoritması çalıştırılarak 
w ve bias değerleri güncellenir. Yani optimizasyon algoritması her batch işlemden sonra devreye sokulmaktadır. Batch büyüklüğü 
fit metodunda batch_size parametresiyle belirtilmektedir. Bu değer girilmezse batch_size 32 olarak alınmaktadır. 32 değeri pek 
çok uygulama için uygun bir değerdir. Optimizasyon işleminin satır satır yapılması yerine batch batch yapılmasının iki önemli 
nedeni vardır: Birincisi işlem miktarının azaltılması, dolayısıyla eğitim süresinin kısaltılmasıdır. İkincisi ise "overfitting" 
denilen olumsuz durum için bir önlem oluşturmasıdır. Overfitting hakkında ileride açıklama yapılacaktır. 

fit metodunun diğer önemli parametresi de "epochs" isimli parametredir. eğitim veri kümesinin eğitim sırasında yeniden 
eğitimde kullanılmasına "epoch" işlemi denilmektedir. Örneğin elimizde 1000 satırlık bir eğitim veri kümesi olsun. batch_size 
parametresinin de 20 olduğunu varsayalım. Bu durumda bu eğitim veri kümesi 1000 / 20 = 50 batch işleminde bitecektir. Yani 
model parametreleri 50 kere ayarlanacaktır. Pek çok durumda eğitim veri kümesinin bir kez işleme sokulması model parametrelerinin
iyi bir biçimde konumlandırılması için yetersiz kalmaktadır. İşte eğitim veri kümesinin birden fazla kez yani fit metodundaki 
epochs sayısı kadar yeniden eğitimde kullanılması yoluna gidilmektedir. Pekiyi epochs değeri ne olmalıdır? Aslında bunu uygulamacı 
belirler. Az sayıda epoch model parametrelerini yeterince iyi konumlandıramayabilir. Çok fazla sayıda epoch "overfitting" 
denilen olumsuz duruma zemin hazırlayabilir. Ayrıca çok fazla epoch eğitim zamanını da uzatmaktadır. Uygulamacı epoch'lar 
sırasında modelin davranışına bakabilir ve uygun epoch sayısında işlemi kesebilir. Eğitim sırasında Keras bizim belirlediğimiz 
fonksiyonları çağırabilmektedir. Buna Keras'ın "callback" mekanizması denilmektedir. Uygulamacı bu yolla model belli bir 
duruma geldiğinde eğitim işlemini kesebilir. Ya da uygulamacı eğer eğitim çok uzamayacaksa yüksek bir epoch ile eğitimini 
yapabilir. İşlemler bitince epoch'lardaki performansa bakabilir. Olması gerekn epoch değerini kestirebilir. Sonra modeli 
yeniden bu sayıda epoch ile eğitir. fit metodunun shuffle parametresi her epoch'tan sonra eğitim veri kümesinin karıştırılıp 
karıştırılmayacağını belirtmektedir. Bu parametre default olarak True biçimdedir. Yani eğitim sırasında her epoch'ta eğitim 
veri kümesi karıştırılmaktadır. 
"""